# Text classification using Transformers.
This lab will focus on text classification on the Imdb dataset.
In this lab session, we will use the encoder-based transformer architecture, through the lens of the most famous model: **BERT**.

---

# Introduction

## HuggingFace

We have already experimented with some components provided by the HuggingFace library:
- the `datasets` library,
- the `tokenizer`.

Actually, HuggingFace library provides convenient API to deal with transformer models, like BERT, GPT, etc.  To quote their website: *Transformers provides APIs and tools to easily download and train state-of-the-art pretrained models. Using pretrained models can reduce your compute costs, carbon footprint, and save you the time and resources required to train a model from scratch. Transformers support framework interoperability between PyTorch, TensorFlow, and JAX.*

## Goal of the lab session

We will experiment with the HuggingFace library. You'll have to load a model and to run it on your task.

Important things to keep in in minds are:
- Even if each model is a Transformer, they all have their peculiarities.
- What is the exact input format expected by the model?
- What is its exact output?
- Can you use the available model as is or should you make some modifications for your task?

These questions are actually part of the life of a NLP scientist. We will adress some of these questions in this lab and in the next lessons / labs / HW.

In [ ]:
%%capture
%pip install transformers datasets

In [1]:
%matplotlib inline
%config InlineBackend.figure_formats = ['svg']
import matplotlib.pyplot as plt

from transformers import DistilBertTokenizer


import numpy as np
import torch
import torch.nn.functional as F
import torch.nn as nn
import math
from torch.utils.data import DataLoader
from tabulate import tabulate
from datasets import load_dataset

from tqdm.notebook import tqdm

# If the machine you run this on has a GPU available with CUDA installed,
# use it. Using a GPU for learning often leads to huge speedups in training.
# See https://developer.nvidia.com/cuda-downloads for installing CUDA
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
DEVICE

device(type='cpu')

## Download the training data

In [3]:
dataset = load_dataset("scikit-learn/imdb", split="train")
print(dataset)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /home/allauzen/.cache/huggingface/datasets/scikit-learn___csv/scikit-learn--imdb-86420cf3ee438711/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.
Dataset({
    features: ['review', 'sentiment'],
    num_rows: 50000
})


## Prepare model inputs

The input format to BERT looks like it is  "over-specified", especially if you focus on just one type task: sequence classification, word tagging, paraphrase detection, etc. The format:
- Add special tokens to the start and end of each sentence.
- Pad & truncate all sentences to a single constant length.
- Explicitly differentiate real tokens from padding tokens with the "attention mask".

It looks like that:

<img src="https://drive.google.com/uc?export=view&id=1cb5xeqLu_5vPOgs3eRnail2Y00Fl2pCo" width="600">

If you don't want to recreate this kind of inputs with your own hands, you can use the pre-trained tokenizer associated to BERT. Moreover the tokenizer will:
- Tokenize the sentence.
- Prepend the `[CLS]` token to the start.
- Append the `[SEP]` token to the end.
- Map tokens to their IDs.
- Pad or truncate the sentence to `max_length`
- Create attention masks for `[PAD]` tokens.


> 💡 *Note:* For computational reasons, we will use the [DistilBERT](https://huggingface.co/docs/transformers/model_doc/distilbert) model, which is a 40% smaller than the original BERT model but still achieve about 95% of the performances of the original model.

In [4]:
tokenizer = DistilBertTokenizer.from_pretrained(
    "distilbert-base-uncased", do_lower_case=True
)

Let's see how the tokenizer actually process the sequence:

In [5]:
# Some useful steps:
message = "hello my name is kevin"
tok = tokenizer.tokenize(message)
print("Tokens in the sequence:", tok)
enc = tokenizer.encode(tok)
table = np.array(
    [
        enc,
        [tokenizer.ids_to_tokens[w] for w in enc],
    ]
).T
print("Encoded inputs:")
print(tabulate(table, headers=["Token IDs", "Tokens"], tablefmt="fancy_grid"))

Tokens in the sequence: ['hello', 'my', 'name', 'is', 'kevin']
Encoded inputs:
╒═════════════╤══════════╕
│   Token IDs │ Tokens   │
╞═════════════╪══════════╡
│         101 │ [CLS]    │
├─────────────┼──────────┤
│        7592 │ hello    │
├─────────────┼──────────┤
│        2026 │ my       │
├─────────────┼──────────┤
│        2171 │ name     │
├─────────────┼──────────┤
│        2003 │ is       │
├─────────────┼──────────┤
│        4901 │ kevin    │
├─────────────┼──────────┤
│         102 │ [SEP]    │
╘═════════════╧══════════╛


🚧 **Question** 🚧

You noticed special tokens like `[CLS]` and `[SEP]` in the sequence. Note how they were added automatically by HuggingFace.

- Why are there such special tokens?

**Answer**

TODO

🚧 **TODO** 🚧

- Run the code below to make sure you can control this behavior

In [13]:
text = "my name is kevin"
tokenized_text_without_special_tokens = tokenizer.encode(text, add_special_tokens=False)
assert tokenized_text_without_special_tokens[0] != 101
print(tokenized_text_without_special_tokens)

[2026, 2171, 2003, 4901]


## Data pre-processing

Usual data-processing for torch. Same as previous lab.

In [14]:
def preprocessing_fn(x, tokenizer):
    x["input_ids"] = tokenizer.encode(
        x["review"],
        add_special_tokens=False,
        truncation=True,
        max_length=256,
        padding=False,
        return_attention_mask=False,
    )
    x["labels"] = 0 if x["sentiment"] == "negative" else 1
    return x

In [16]:
n_samples = 2000  # the number of training example

# We first shuffle the data !
dataset = dataset.shuffle()

# Select n_samples
splitted_dataset = dataset.select(range(n_samples))

# Tokenize the dataset
splitted_dataset = splitted_dataset.map(
    preprocessing_fn, fn_kwargs={"tokenizer": tokenizer}
)


# Remove useless columns
splitted_dataset = splitted_dataset.select_columns(["input_ids", "labels"])

# Split the train and validation
splitted_dataset = splitted_dataset.train_test_split(test_size=0.2)

train_set = splitted_dataset["train"]
valid_set = splitted_dataset["test"]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [17]:
class DataCollator:
    def __init__(self, tokenizer):
        self.tokenizer = tokenizer

    def __call__(self, batch):
        # `batch` is a list of dictionary with keys "review_ids" and "label".
        features = self.tokenizer.pad(
            batch, padding="longest", max_length=256, return_tensors="pt"
        )
        return features

In [18]:
data_collator = DataCollator(tokenizer)

In [19]:
batch_size = 4

train_dataloader = DataLoader(
    train_set, batch_size=batch_size, collate_fn=data_collator
)
valid_dataloader = DataLoader(
    valid_set, batch_size=batch_size, collate_fn=data_collator
)
n_valid = len(valid_set)
n_train = len(train_set)

# Model from scratch

For this first exercise, we will start from a randomly initialized model.

## Retrieve the architecture configuration

In HuggingFace, model's parameters are specified through a `config` file. It is a json-like object.

We can retrieve the one from the official model with the following code:

In [20]:
from transformers import DistilBertConfig

model_config = DistilBertConfig.from_pretrained("distilbert-base-uncased")
print(model_config)

DistilBertConfig {
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.29.2",
  "vocab_size": 30522
}



🚧 **Question** 🚧

Make sure you understand the parameters of the configuration.
- Which ones are task-agnostic parameters?
- Which ones are not?
- Why are there different parameters for different tasks?

**Answer**

TODO



Several architectures are available for DistilBert on HuggingFace, designed for a variety of NLP tasks. Though these interfaces are all built on top of a trained DistilBert model, each has different top layers and output types designed to accomodate their specific NLP task.  

Here is the current list of classes provided for fine-tuning:
* BertModel
* BertForMaskedLM
* BertForNextSentencePrediction
* BertForSequenceClassification
* BertForTokenClassification
* BertForQuestionAnswering

The documentation for these can be found under [here](https://huggingface.co/docs/transformers/model_doc/distilbert).




🚧 **TODO** 🚧

For our first experiment, we want to build from a standard stack of transformer layers, without any additional task-specific head.

Which architecture is the corresponding one?

Choose the right one and initialize the model below, with the config.

In [ ]:
# from transformers import # TODO

# bert = # TODO

In [22]:
print(bert)

DistilBertModel(
  (embeddings): Embeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer): Transformer(
    (layer): ModuleList(
      (0): TransformerBlock(
        (attention): MultiHeadSelfAttention(
          (dropout): Dropout(p=0.1, inplace=False)
          (q_lin): Linear(in_features=768, out_features=768, bias=True)
          (k_lin): Linear(in_features=768, out_features=768, bias=True)
          (v_lin): Linear(in_features=768, out_features=768, bias=True)
          (out_lin): Linear(in_features=768, out_features=768, bias=True)
        )
        (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (ffn): FFN(
          (dropout): Dropout(p=0.1, inplace=False)
          (lin1): Linear(in_features=768, out_features=3072, bias=True)
          (lin2): Linear(i

Just for curiosity's sake, we can browse all of the model's parameters by name here.

In the cell below, we printed out the names and dimensions of the weights for:

- The embedding layer
- The first of the twelve transformers
- The output layer.



In [26]:
# Get all of the model's parameters as a list of tuples.
params = list(bert.named_parameters())

In [27]:
print("The BERT model has {:} different named parameters.\n".format(len(params)))

print("==== Embedding Layer ====\n")

for p in params[0:4]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print("\n==== First Transformer Layer ====\n")

for p in params[4:20]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

The BERT model has 100 different named parameters.

==== Embedding Layer ====

embeddings.word_embeddings.weight                       (30522, 768)
embeddings.position_embeddings.weight                     (512, 768)
embeddings.LayerNorm.weight                                   (768,)
embeddings.LayerNorm.bias                                     (768,)

==== First Transformer Layer ====

transformer.layer.0.attention.q_lin.weight                (768, 768)
transformer.layer.0.attention.q_lin.bias                      (768,)
transformer.layer.0.attention.k_lin.weight                (768, 768)
transformer.layer.0.attention.k_lin.bias                      (768,)
transformer.layer.0.attention.v_lin.weight                (768, 768)
transformer.layer.0.attention.v_lin.bias                      (768,)
transformer.layer.0.attention.out_lin.weight              (768, 768)
transformer.layer.0.attention.out_lin.bias                    (768,)
transformer.layer.0.sa_layer_norm.weight                 

🚧 **TODO** 🚧

Test your `bert`.
We can already try the model on the validation set. Before just look at the output of the model on one batch.
- Interpret the output.
- Do you understand everything ?


In [29]:
batch = next(iter(train_dataloader))

input_ids = batch["input_ids"]
attention_mask = batch["attention_mask"]

cprint(output["last_hidden_state"].shape)
print(type(output))
print(output)

torch.Size([4, 256, 768])
<class 'transformers.modeling_outputs.BaseModelOutput'>
BaseModelOutput(last_hidden_state=tensor([[[-1.3653,  1.2220, -0.3781,  ..., -0.3138, -1.8206,  0.4190],
         [ 0.0401, -1.1929, -0.5043,  ..., -1.4991, -0.7347, -1.9784],
         [-1.1681,  1.1502,  0.7346,  ..., -0.8591,  0.2382,  0.5894],
         ...,
         [-0.1164, -0.7715, -1.1786,  ...,  0.2194, -0.4574, -0.6372],
         [ 1.6932, -0.1928,  1.8059,  ...,  0.5969, -0.4372, -0.4528],
         [-1.4921,  0.4403,  0.1746,  ...,  0.8265,  1.9190,  0.6501]],

        [[-2.2565,  2.2618,  0.3376,  ...,  0.8793, -1.1041, -0.1474],
         [ 1.0984,  0.1403, -0.0570,  ..., -0.5157, -0.6845,  0.2363],
         [ 0.2041, -0.2521,  0.0882,  ..., -1.7599, -0.2288, -1.0007],
         ...,
         [-1.0553, -1.0914, -0.8090,  ...,  0.4667, -0.7112,  0.4885],
         [ 0.1955,  0.8297, -0.0910,  ...,  0.4149, -0.5729,  0.0541],
         [-2.0361,  1.2245, -0.1805,  ...,  1.0892,  1.1073,  0.5746]],



## Building a classifier

Our `bert` model is simply a stack of transformer layers. We would like to use it as a backbone for text classification.

🚧 **TODO** 🚧

Wraps the model into a classifier.

> 💡 *Hint*: Use the last hidden [CLS] vector representation to perform classification.

In [ ]:
class DistilBertClassifier(nn.Module):
    def __init__(self, bert, dropout=0.1):
        self.bert = bert
        self.dp = nn.Dropout(dropout)
        self.classifier = nn.Linear(768, 1)

    def forward(self, input_ids, attention_mask):
        bert_out = self.bert(input_ids, attention_mask)
        # out = # TODO
        # out = # TODO
        return self.classifier(out)

In [33]:
bert = DistilBertModel(model_config)
model = DistilBertClassifier(bert)
model.to(DEVICE)

DistilBertClassifier(
  (bert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
            (lin1): Linear(i

🚧 **TODO** 🚧

Test your model on the batch.
Make sure it has the right shape.

In [35]:
out = model.forward(input_ids, attention_mask)
print(out)

tensor([[ 1.2068],
        [ 0.5691],
        [-0.2065],
        [ 0.1523]], grad_fn=<AddmmBackward0>)


### Training

🚧 **TODO** 🚧

Train your model.
Make sure you track the following quantities per epoch:
- training loss
- training accuracy
- validation loss
- validation accuracy

In [38]:
# Redefine the dataloaders to adjust the batch size.
batch_size = 32

train_dataloader = DataLoader(
    train_set, batch_size=batch_size, collate_fn=data_collator
)
valid_dataloader = DataLoader(
    valid_set, batch_size=batch_size, collate_fn=data_collator
)
n_valid = len(valid_set)
n_train = len(train_set)

In [ ]:
def validation(model, valid_dataloader):
    total_size = 0
    acc_total = 0
    loss_total = 0
    criterion = nn.BCEWithLogitsLoss()
    model.eval()
    with torch.no_grad():
        for batch in tqdm(valid_dataloader):
            batch = {k: v.to(DEVICE) for k, v in batch.items()}
            input_ids = batch["input_ids"]
            labels = batch["labels"]
            attention_mask = batch["attention_mask"]
            labels = labels.float()
            preds = model(input_ids=input_ids, attention_mask=attention_mask)
            loss = criterion(preds.squeeze(), labels)
            acc = (preds.squeeze() > 0) == labels
            total_size += acc.shape[0]
            acc_total += acc.sum().item()
            loss_total += loss.item()
    model.train()
    return loss_total / len(valid_dataloader), acc_total / total_size


validation(model, valid_dataloader)

  0%|          | 0/13 [00:00<?, ?it/s]

(0.7154629826545715, 0.4975)

In [41]:
def training(model, n_epochs, train_dataloader, valid_dataloader, lr=5e-5):
    optimizer = torch.optim.AdamW(
        model.parameters(),
        lr=lr,
        eps=1e-08,
    )
    list_val_acc = []
    list_train_acc = []
    list_train_loss = []
    list_val_loss = []
    criterion = nn.BCEWithLogitsLoss()
    for e in range(n_epochs):
        # ========== Training ==========

        # Set model to training mode
        model.train()
        model.to(DEVICE)

        # Tracking variables
        train_loss = 0
        epoch_train_acc = 0
        for batch in tqdm(train_dataloader):
            batch = {k: v.to(DEVICE) for k, v in batch.items()}
            input_ids, attention_mask, labels = (
                batch["input_ids"],
                batch["attention_mask"],
                batch["labels"],
            )
            optimizer.zero_grad()
            # Forward pass
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            # Backward pass
            loss = criterion(outputs.squeeze(), labels.squeeze().float())
            loss.backward()
            optimizer.step()
            train_loss += loss.detach().cpu().item()
            acc = (outputs.squeeze() > 0) == labels.squeeze()
            epoch_train_acc += acc.float().mean().item()
        list_train_acc.append(100 * epoch_train_acc / len(train_dataloader))
        list_train_loss.append(train_loss / len(train_dataloader))

        # ========== Validation ==========

        l, a = validation(model, valid_dataloader)
        list_val_loss.append(l)
        list_val_acc.append(a * 100)
        print(
            e,
            "\n\t - Train loss: {:.4f}".format(list_train_loss[-1]),
            "Train acc: {:.4f}".format(list_train_acc[-1]),
            "Val loss: {:.4f}".format(l),
            "Val acc:{:.4f}".format(a * 100),
        )
    return list_train_loss, list_train_acc, list_val_loss, list_val_acc

In [ ]:
training(model, 3, train_dataloader, valid_dataloader)

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/13 [00:00<?, ?it/s]

0 
	 - Train loss: 0.7599 Train acc: 49.6875 Val loss: 0.6836 Val acc:53.0000


  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/13 [00:00<?, ?it/s]

1 
	 - Train loss: 0.7105 Train acc: 54.2500 Val loss: 0.6638 Val acc:56.7500


  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/13 [00:00<?, ?it/s]

2 
	 - Train loss: 0.5527 Train acc: 71.0000 Val loss: 0.4880 Val acc:76.5000


([0.7598638701438903, 0.7105308413505554, 0.5526753580570221],
 [49.6875, 54.25, 71.0],
 [0.6836349780742939, 0.6637736742313092, 0.4879949207489307],
 [53.0, 56.75, 76.5])

## Pre-trained model

Now we are going to compare with a pre-trained model.

First, we are going to load the model's weights from the HuggingFace hub.

In [44]:
bert = DistilBertModel.from_pretrained("distilbert-base-uncased")

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.weight', 'vocab_projector.bias', 'vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


## Fine-Tuning

With our model loaded and ready,  we need to grab the training hyperparameters from within the stored model.

For the purposes of fine-tuning, the authors recommend choosing from the following values (from Appendix A.3 of the [BERT paper](https://arxiv.org/pdf/1810.04805.pdf)):

- Batch size: 16, 32  
- Learning rate (Adam): 5e-5, 3e-5, 2e-5  
- Number of epochs: 2, 3, 4

We chose:
* **Batch size**: 16 (set when creating our DataLoaders)
* **Learning rate**: 5e-5
* **Epochs**: 3 (we'll see that this is probably too many...)

The epsilon parameter `eps = 1e-8` is "a very small number to prevent any division by zero in the implementation" (from [here](https://machinelearningmastery.com/adam-optimization-algorithm-for-deep-learning/)).

You can find the creation of the AdamW optimizer in `run_glue.py` [here](https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L109).

🚧 **TODO** 🚧

Build the classifier and train it with the pre-trained checkpoint.

In [ ]:
# TODO

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

0 
	 - Train loss: 0.2763 Train acc: 90.6250 Val loss: 0.7553 Val acc:76.0000


  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

...

  0%|          | 0/7 [00:00<?, ?it/s]

3 
	 - Train loss: 0.0068 Train acc: 99.7500 Val loss: 1.0071 Val acc:78.0000


  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

4 
	 - Train loss: 0.0028 Train acc: 100.0000 Val loss: 1.0061 Val acc:78.0000


🚧 **Question** 🚧

What do you think of the results?

**Answer**

TODO

## Pre-built models

Actually, you built your own classifier based on the raw output of a transformers, wrapped into a classification model.

But, for many tasks, you can directly download the model with the necessary blocks.

For instance, we could directly have loaded `DistilBertForSequenceClassification`.

Let's see the difference with our model.

🚧 **TODO** 🚧

Load a pre-trained `DistilBertForSequenceClassification`.

In [32]:
model = # TODO
print(model)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

🚧 **Question** 🚧

Here there might be a lot of questions:
- what does the warning means?
- why `num_labels=1`?
- and the other options?

**Answer**

TODO

🚧 **Question** 🚧

How is the classifcation made? Is it the same than with our model? You might need to check the official implementation [here](https://github.com/huggingface/transformers/blob/v4.34.1/src/transformers/models/distilbert/modeling_distilbert.py#L730).

**Answer**

TODO

🚧 **TODO** 🚧

The output of such a model is not directly the logits, but a wrapper that can return several objects. Analyze it and modify the training and validation loop accordingly.

Launch the training.

Do you observe any differences with our own classification model?

In [33]:
# TODO

# Interpretability

So far we have models able to predict quite faithfully if a critic is positive or negative. But can we interprete the results?

A usual way to do so with transformers, is simply to look at the **attention weights**. Let's see if we can get some insights on the model's prediction using this technique.


First, tokenize the text.

In [108]:
text = (
    "captain corelli's mandolin is a beautiful film with a lovely cast"
    " including the wonderful nicolas cage, who as always is brilliant in the movie."
    " the music in the film is really nice too. i'd advise anyone to go and see it. brilliant! 10 / 10 "
)
tokenized_text = tokenizer(text, return_tensors="pt")

🚧 **TODO** 🚧

Now feed it as an output for the model. Use the keyword-argument `output_attentions=True`.


In [109]:
model_output = # TODO

🚧 **TODO** 🚧 
Check the prediction and plot the attention weights matrix.

In [ ]:
# TODO

🚧 **Question** 🚧

Is your interpretability experiment conclusive?